In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
import cv2

In [9]:
from keras.preprocessing import image
from data_preprocessor.PreProcessImgData import PreProcessImgData

In [7]:
# image_path_samples = '../train_camVid/*.png'
# image_path_labels = '../LabeledApproved_full/*.png'
image_path_samples = 'C:/Krishna/DataSets/cam_vid_semantic/train_samples/*.png'
image_path_labels = 'C:/Krishna/DataSets/cam_vid_semantic/train_labels/*.png'

target_width = 256
target_height = 256
labels_dict = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128], [128, 0, 128]]

In [10]:
p = PreProcessImgData(image_path_samples,
                      image_path_labels,
                      len(labels_dict), target_width, target_height, labels_dict)

In [12]:
from sklearn.model_selection import train_test_split

train_samples,test_samples,train_labels,test_labels = train_test_split(p.file_list_samples,p.file_list_labels,test_size=0.33, random_state=42)

In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [4]:
## Model for FCNNs with skip connections

img_input = Input(shape=(256,256,3))
conv1 = Conv2D(64,(3,3),activation= 'relu',padding='same')(img_input)
pool1 = MaxPool2D(pool_size=(2,2),padding='same')(conv1)

conv2 = Conv2D(128,(3,3),activation= 'relu',padding='same')(pool1)
pool2 = MaxPool2D(pool_size=(2,2),padding='same')(conv2)

conv3 = Conv2D(256,(3,3),activation= 'relu',padding='same')(pool2)
drop3 = Dropout(0.3)(conv3)
pool3 = MaxPool2D(pool_size=(2,2),padding='same')(drop3)

conv4 = Conv2D(512,(3,3),activation= 'relu',padding='same')(pool3)
drop4 = Dropout(0.3)(conv4)

# up_sample4 = UpSampling2D((2,2))(drop4)
# conv_transpose4 = Conv2DTranspose(512,(3,3),activation='relu',padding='same')(up_sample4)
# skip_connected_4 = Add()([conv_transpose4,drop4])

up_sample3 = UpSampling2D((2,2))(drop4)
conv_transpose3 = Conv2DTranspose(256,(3,3),activation='relu',padding='same')(up_sample3)
skip_connected_3 = Add()([conv_transpose3,drop3])

up_sample2 = UpSampling2D((2,2))(skip_connected_3)
conv_transpose2 = Conv2DTranspose(128,(3,3),activation='relu',padding='same')(up_sample2)
skip_connected_2 = Add()([conv_transpose2,conv2])

up_sample1 = UpSampling2D((2,2))(skip_connected_2)
conv_transpose1 = Conv2DTranspose(64,(3,3),activation='relu',padding='same')(up_sample1)
out_layer = Conv2D(20,(1,1),activation='softmax',padding='same')(conv_transpose1)


model = Model(input = img_input,output = out_layer)

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 128, 128, 128 73856       max_pooling2d_7[0][0]            
__________________________________________________________________________________________________
max_poolin

C:\Users\nadim\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [ ]:
model.compile(optimizer = 'sgd', loss = 'mse', metrics = ['accuracy'])

In [ ]:
check_pointer = ModelCheckpoint('best_weights/best_weights.hdf5',verbose=1,save_best_only=True)
history = model.fit(train_samples_array,train_labels_array,epochs=5,verbose=1,validation_split=0.1, batch_size=5,callbacks=[check_pointer])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# model.fit(train_samples_array,train_labels_array,epochs=5,verbose=2,batch_size=5,validation_split=0.2,callbacks=[check_pointer])

In [ ]:
test_metrics = model.evaluate(test_samples_array,test_labels_array)

print(test_metrics)

In [ ]:
pred_array = model.predict(np.expand_dims(train_samples_array[20],axis=0))
label_array =  train_labels_array[20]

In [ ]:
imshow(Image.fromarray(np.squeeze((pred_array*255).astype('uint8'))))


In [ ]:
imshow(Image.fromarray(np.squeeze((label_array*255).astype('uint8'))))